## Find inverse kinematic angles using scipi.optimize

In [2]:
import numpy as np
from nicos_FwdKinArmRob_serial import armrobfwdkin

# testing forward kinematics. Seems to work. 
test_angles = [0.244978663127, -0.553231565311, 1.98924862896, 0.0, -1.43601706365, 0.0]
test_angles = [0.0, -np.pi/2., np.pi/2., 0.0, 0.0, 0.0]
test_endpoint = armrobfwdkin(test_angles)
test_endpoint


array([0.2057, 0.    , 0.2473])

In [23]:
import scipy.optimize as optimize
import numpy as np
from nicos_FwdKinArmRob_serial import armrobfwdkin

def endpoint_error(angles, args):
    true_endpoint = args
    x_true = true_endpoint[0]
    y_true = true_endpoint[1]
    alpha = np.arctan2(y_true, x_true)
    beta1 = angles[0]
    beta2 = angles[1]
    beta3 = angles[2]
    gamma3 = 0.0
    gamma5 = 0.0
    all_angles = [alpha, beta1, beta2, gamma3, beta3, gamma5]

    estimated_endpoint = armrobfwdkin(all_angles)  # in radians 
    diff = true_endpoint - estimated_endpoint
    error = np.linalg.norm(diff)
    return error

def calculate_angles(endpoint,angle_guess):
    bounds = [(-3*np.pi/4, 0),  # Bounds for beta1
            (0, np.pi),  # Bounds for beta2
            (0, np.pi/2)]  # Bounds for beta3. have gripper above the pieces

    # Perform the optimization with bounds
    calculated_beta_angles = optimize.minimize(endpoint_error, angle_guess, args=(endpoint), method="SLSQP", bounds=bounds)

    return calculated_beta_angles

def nico_IK(endpoint):
    angle_guess = [-np.pi/4, np.pi/2, np.pi/4]
    beta_angles = calculate_angles(endpoint, angle_guess).x
    alpha = np.arctan2(endpoint[1], endpoint[0])
    angles = [alpha, beta_angles[0], beta_angles[1], 0, beta_angles[2], 0]
    return np.asfarray(angles)

# Testing
endpoint = [0.2057, 0.1, 0.2473]
endpoint = [0.1,0,0.07]
angle_guess = [0, 0, 0]
# calculate_angles(endpoint, angle_guess).success # returns 'true'
np.degrees(nico_IK(endpoint))


array([  0.        , -64.8054255 ,  97.65012955,   0.        ,
        76.3541811 ,   0.        ])

### This method seems to work. Now I'll test it on a number of test points associated to the chess board

In [24]:
from chessboard_utils import make_move_path, load_config

start_square = 'A1'
end_square_list = [f"{chr(row)}{col}" for row in range(ord('A'), ord('H') + 1) for col in range(1, 9)]

config = load_config('robot.yaml')
move_paths = []
for end_square in end_square_list:
    move_paths.append(make_move_path(start_square, end_square, config))

successes = []
path = move_paths[0] # change this index (up to 63) to test different moves
for point in path:
    success = calculate_angles(point,angle_guess=[-np.pi/4, np.pi/2, np.pi/4]).success
    successes.append(success)

successes

[True, True, True, True, True, True, True, True]

### Looks like we're able to solve IK for several different chess moves. Now we should visualize with a sim or the real robot.

In [25]:

endpoint = [0.2057, 0.1, 0.2473]
#endpoint = [0.1, 0, 0.05]
# calculate_angles(endpoint, angle_guess).success # returns 'true'
calc_angles = nico_IK(endpoint)
armrobfwdkin(calc_angles)

array([0.20570083, 0.10000041, 0.24730118])